# 👑 Mobile Price 예측 모델링 (분류 과제 편)
- 빅데이터 분석기사 실기 연습을 위해 제가 만든 자료입니다.
- 데이터셋은 kaggle의 데이터셋을 제가 다시 가공한 뒤 X_train, y_train, X_test로 분리하였습니다. 원본 데이터셋은 다음의 주소에서 확인하실 수 있습니다 : (https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification/data?select=train.csv) 

### 👑 문제
- 주어진 학습용 데이터 X_train.csv를 활용하여 모바일 폰의 가격 범위(price_range)를 예측하는 모형을 만들고, 평가용 데이터 X_test.csv에 적용하여 가격 범위 예측값을 확률로 만들어 csv 파일로 생성하시오.(평가 지표는 ROC-AUC)

### 👑 힌트
- 분류 과제인지 회귀 과제인지 우선 파악해야 합니다.
- 평가 지표로 ROC-AUC를 사용하라고 한 것으로 보아 분류과제입니다. 이에 맞는 예측 모형으로 분류에 적합한 모형을 선정해야 합니다.
- 분류 과제에서는 pred와 proba가 모두 사용될 수 있으므로 무엇을 도출해야 하는지 파악합니다. 여기서는 확률을 요구하고 있으므로 proba를 사용합니다. 
- proba로 확률값을 요구하는 경우에는 평가지표로 ROC-AUC가 사용됩니다.

# 1. 필요한 패키지 불러오기
- 기본적인 패키지들을 불러옵니다.

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# 2. 데이터 파일 읽기
- 시험에는 총 3개의 파일이 주어집니다.
- X_train : 학습 feature로 사용되는 데이터입니다. 
- X_test : 평가를 위해 사용되는 데이터입니다.
- y_train : 타겟으로 사용되는 데이터입니다.

In [2]:
x_train = pd.read_csv("./Mobile_price_X_train.csv")
x_test = pd.read_csv("./Mobile_price_X_test.csv")
y_train = pd.read_csv("./Mobile_price_y_train.csv")

In [3]:
x_train

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w
0,842.0,2.2,1,7,0.6,188,2,2,20,756,2549,9,7
1,1821.0,1.2,13,44,0.6,141,2,14,1208,1212,1411,8,2
2,1859.0,0.5,3,22,0.7,164,1,7,1004,1654,1067,17,1
3,1954.0,0.5,0,24,0.8,187,4,0,512,1149,700,16,3
4,1445.0,0.5,0,53,0.7,174,7,14,386,836,1099,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,586.0,2.8,2,15,0.2,83,3,11,241,854,2592,12,8
996,1617.0,2.4,8,36,0.8,85,1,9,743,1426,296,5,3
997,674.0,2.9,1,21,0.2,198,3,4,576,1809,1180,6,3
998,794.0,0.5,0,2,0.8,106,6,14,1222,1890,668,13,4


In [4]:
y_train

,price_range
0,1
1,1
2,1
3,0
4,0
...,...
995,1
996,0
997,0
998,0


# 3. info() 함수를 통해 데이터에 대한 기본 정보 확인
- 데이터의 행의 수와 열의 수를 확인합니다.
- 결측치를 확인하고 만약 결측치가 존재한다면 이를 보간하거나 삭제하는 등의 방법으로 처리해야 합니다.

In [5]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  998 non-null    float64
 1   clock_speed    1000 non-null   float64
 2   fc             1000 non-null   int64  
 3   int_memory     1000 non-null   int64  
 4   m_dep          1000 non-null   float64
 5   mobile_wt      1000 non-null   int64  
 6   n_cores        1000 non-null   int64  
 7   pc             1000 non-null   int64  
 8   px_height      1000 non-null   int64  
 9   px_width       1000 non-null   int64  
 10  ram            1000 non-null   int64  
 11  sc_h           1000 non-null   int64  
 12  sc_w           1000 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 101.7 KB


In [6]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  500 non-null    int64  
 1   clock_speed    500 non-null    float64
 2   fc             500 non-null    int64  
 3   int_memory     500 non-null    int64  
 4   m_dep          500 non-null    float64
 5   mobile_wt      500 non-null    int64  
 6   n_cores        500 non-null    int64  
 7   pc             500 non-null    int64  
 8   px_height      500 non-null    int64  
 9   px_width       500 non-null    int64  
 10  ram            500 non-null    int64  
 11  sc_h           500 non-null    int64  
 12  sc_w           500 non-null    int64  
dtypes: float64(2), int64(11)
memory usage: 50.9 KB


# 4. 결측치 처리
- X_train.csv에는 "battery_power"라는 컬럼에 결측치가 존재합니다. 여기서는 이 결측치를 중앙값으로 보간하는 방법으로 처리하겠습니다.

In [7]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
x_train[['battery_power']] = imputer.fit_transform(x_train[['battery_power']])

In [8]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1000 non-null   float64
 1   clock_speed    1000 non-null   float64
 2   fc             1000 non-null   int64  
 3   int_memory     1000 non-null   int64  
 4   m_dep          1000 non-null   float64
 5   mobile_wt      1000 non-null   int64  
 6   n_cores        1000 non-null   int64  
 7   pc             1000 non-null   int64  
 8   px_height      1000 non-null   int64  
 9   px_width       1000 non-null   int64  
 10  ram            1000 non-null   int64  
 11  sc_h           1000 non-null   int64  
 12  sc_w           1000 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 101.7 KB


# 5. describe() 함수를 통해 요약 통계량 확인
- describe() 함수는 각 열에 대한 통계량을 보여줍니다.
- 여기서는 이상치를 확인하고자 합니다. 
- 예를 들어, 평균 및 표준편차를 고려할 때 max값이 매우 클 때 이상치로 판단합니다. 
- 아래의 데이터셋에는 특별한 이상치는 보이지 않는 것으로 판단됩니다.

In [9]:
x_train.describe()

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1173.241000,1.519400,4.212000,31.645000,0.507100,140.531000,4.449000,9.749000,601.650000,1201.089000,1232.402000,12.268000,5.613000
std,428.167851,0.832043,4.344183,18.048067,0.287893,36.012302,2.294049,6.109773,413.507064,426.239324,611.732959,4.215204,4.205106
min,501.000000,0.500000,0.000000,2.000000,0.100000,80.000000,1.000000,0.000000,0.000000,500.000000,256.000000,5.000000,0.000000
25%,808.750000,0.600000,1.000000,16.000000,0.200000,109.000000,2.000000,4.000000,277.500000,831.000000,703.750000,9.000000,2.000000
50%,1138.000000,1.500000,3.000000,31.000000,0.500000,141.000000,4.000000,10.000000,518.500000,1185.000000,1209.500000,12.000000,5.000000
75%,1522.250000,2.300000,7.000000,47.000000,0.800000,172.000000,6.000000,15.000000,882.250000,1534.500000,1699.500000,16.000000,8.000000
max,1996.000000,3.000000,19.000000,64.000000,1.000000,200.000000,8.000000,20.000000,1914.000000,1998.000000,2811.000000,19.000000,18.000000


# 6. 수치형 변수와 명목형 변수를 분리
- 이 데이터셋은 모두 수치형 변수로만 이루어져 있습니다. 그러나 명목형 변수가 존재하는 데이터셋도 있으므로 이러한 변수들을 각 리스트에 분리하는 것이 도움이 됩니다.

In [10]:
col_number = ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 
              'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w']
col_category = []
col_y = ['price_range']

# 7. 학습 데이터와 검증 데이터로 학습 데이터를 다시 분할
- 학습 데이터로 만든 모델의 일반화 성능을 검증하기 위해 다시 분할하여 검증합니다.
- 과적합이 발생하는 경우에 모델을 바꾸거나 하이퍼파라미터를 조정하는 등의 방식으로 모델링 방식을 바꿉니다. 
- 여기서 학습 데이터와 검증 데이터를 나누는 비율은 0.3으로 설정하였습니다.
### 💢💥 주의
- 분류과제에서는 stratify 옵션을 사용하는 경우가 많습니다. 클래스의 비율에 따라 훈련-검증 데이터의 비율을 유지하기 위해서는 이 옵션을 반드시 사용해야 합니다.

In [11]:
from sklearn.model_selection import train_test_split

x_tr, x_valid, y_tr, y_valid = train_test_split(x_train, y_train['price_range'], 
                                                test_size = 0.3,
                                                stratify = y_train['price_range'])

# 8. 스케일링 수행
- 스케일링은 데이터의 특성들을 동일한 스케일로 변환하여 모델이 학습하는 데 도움을 줍니다. 
- 각 feature들의 단위가 다르기 때문에 이를 동일한 조건으로 만들어주고자 스케일링을 진행합니다.
- 그리고 스케일링은 학습 데이터에 대해서만 진행하므로 scaler.fit을 사용하여 스케일링 파리미터(평균과 표준편자)를 계산합니다.
- 검증 데이터와 테스트 데이터에 대해서는 scaler.fit()을 호출하지 않고, 학습 데이터에서 계산한 파라미터를 사용하여 scaler.transform()을 호출합니다.
- 아래에서 사용된 StandardScaler는 평균을 0, 표준편차를 1로 맞춰주는 스케일러입니다.

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_tr[col_number])

x_tr[col_number] = scaler.transform(x_tr[col_number])
x_valid[col_number] = scaler.transform(x_valid[col_number])
x_test[col_number] = scaler.transform(x_test[col_number])

# 9. 모델 학습
- 분류 과제를 수행할 수 있는 모델의 종류는 매우 다양합니다.
- 시험에서는 랜덤 포레스트를 사용하는 것이 유리한 경우가 많습니다.
- 랜덤 포레스트는 분류 과제와 회귀 과제(RandomForestRegressor)에 모두 사용될 수 있고, 데이터에 따라 다르지만 대체로 모델의 성능도 우수한 편입니다.

In [13]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(random_state = 42)
modelRF.fit(x_tr, y_tr)

RandomForestClassifier(random_state=42)

# 10. 모델 평가
- 검증 데이터 X_valid에 대한 예측을 수행한 뒤, 이 예측 결과를 실제 검증 데이터의 라벨(y_valid)와 비교하여 점수를 계산합니다.

In [14]:
y_validation_pred_proba = modelRF.predict_proba(x_valid)

In [15]:
from sklearn.metrics import roc_auc_score

scoreRF = roc_auc_score(y_valid, y_validation_pred_proba[:, 1])

In [16]:
print(scoreRF)

0.9779333333333332


- 랜덤 포레스트 모델(modelRF)을 사용하여 테스트 데이터(x_test)에 대한 클래스 예측 확률을 계산합니다.

In [17]:
pred = modelRF.predict_proba(x_test)[:, 1]

# 11. 파일 제출
- 시험에서 요구하는 형태로 csv 파일을 제출합니다. index = False로 지정해서 새로운 인덱스가 포함되지 않도록 설정합니다.
- 만들어진 최종 제출본을 다시 한번 읽어 제대로 만들어졌는지 내용을 확인합니다.

In [18]:
pd.DataFrame({'submission':pred}).to_csv('mobile_submission.csv', index = False)

In [19]:
df = pd.read_csv('./mobile_submission.csv')
df

,submission
0,0.77
1,0.90
2,0.10
3,0.89
4,0.92
...,...
495,0.86
496,0.17
497,0.29
498,0.95
